In [43]:
# Download the basics
from astropy.utils.data import download_file
from astropy.time import Time, TimeDelta
import xml.etree.ElementTree as ET, gzip, io

from glob import glob

from piaa import exoplanets

from astropy.io import fits
from astropy import units as u
from astropy import constants
from astropy.time import Time, TimeDelta
import numpy as np
import urllib

In [44]:
# Download the OEC file to use
url = "https://github.com/OpenExoplanetCatalogue/oec_gzip/raw/master/systems.xml.gz"
oec_file = download_file(url)

### OEC Class

Wrap the [OpenExoplanetCatalogue](https://github.com/OpenExoplanetCatalogue/open_exoplanet_catalogue) to make it easier to work with in scripts. You can see their [example](https://github.com/OpenExoplanetCatalogue/open_exoplanet_catalogue#how-to-access-the-catalogue-using-python).

We are simply going to make it easier to work with, anticipating we will use it a lot.

### Transit namedtuple

[`namedtuple`s](https://docs.python.org/3.6/library/collections.html#collections.namedtuple) are great for collections of properties about something. Here we make a simple representation of some transit properties

In [46]:
from collections import namedtuple

Transit = namedtuple('Transit', ['ingress', 'midpoint', 'egress'])

#### Base class

We make a base class that will handle all the basic properties that are common to both `Star`s and `Exoplanet`s.

In [47]:
class OEC(object):
    def __init__(self, file=None, *args, **kwargs):
        """ Base object for the OpenExoplanetCatalogue 
    
        The object can load from a local file or will alternatively download
        the lastest catalogue from the OEC.
        """
        if file is None:
            url = "https://github.com/OpenExoplanetCatalogue/oec_gzip/raw/master/systems.xml.gz"
            file = io.BytesIO(urllib.request.urlopen(url).read())
            self.oec = ET.parse(gzip.GzipFile(fileobj=file))        
        else:
            self.oec = ET.parse(gzip.GzipFile(filename=file))
            
        self.names = list()

    def _build_properties(self, element):
        for elem in element:
            if elem.tag != 'name':
                try:
                    setattr(self, elem.tag, float(elem.text))
                except ValueError:
                    setattr(self, elem.tag, elem.text)
                except AttributeError:
                    pass
                except TypeError:
                    pass
            else:
                self.names.append(elem.text)        

    @property
    def name(self):
        return self.names[0]                

#### `Star` class

We inherit from `OEC` and then use the passed `name` lookup the `Star` in the catalogue.

A `Star` will automatically look for any `Exoplanet`s that belong to it.

In [50]:
class Star(OEC):
    def __init__(self, name, *args, **kwargs):
        """ Lookup information about a star system """
        super().__init__(*args, **kwargs)
        
        # Get the star info
        self.system = self.oec.findall(".//system[name='{}']".format(name))[0]
        element = self.system.find('.//star')
        self._build_properties(element)

        # Search for exoplanets
        self.planet = Exoplanet(self.system.find('.//planet'), star=self)

#### `Exoplanet` Class

Again we inherit from `OEC`. The basic properties for an `Exoplanet` are fairly simply but this class provides a lot of methods for working with an `Exoplanet`.

In [51]:
class Exoplanet(OEC):
    def __init__(self, element, star=None, *args, **kwargs):
        """ Object representing an exoplanet
        
        Note that an `Exoplanet` usually belongs to a `Star`
        """
        super().__init__(*args, **kwargs)
        self._build_properties(element)
        self.star = star
        
        self._transit_duration = None
        self._b = None
        self._transit_depth = None
        
    @property
    def transit_duration(self):
        """ Get transit duration in minutes
        
        Seager, S., & Mallen-Ornelas, G. (2002). 
        The Astrophysical Journal, 585, 1038–1055. https://doi.org/10.1086/346105
        """
        if self._transit_duration is None:
            i = self.inclination * u.degree
            r_s = self.star.radius * u.R_sun
            r_p = self.radius * u.R_jup
            period = self.period * u.day
            a = self.semimajoraxis * u.AU
            
            self._transit_duration = ((period / np.pi) * np.arcsin(
            (r_s / a) * 
            np.sqrt(
                (
                    (1 + (r_p / r_s))**2 - 
                        ((a / r_s) * np.cos(i))**2
                ) / (1 - np.cos(i)**2)
            )
        ).value).to(u.minute).value
            
        return self._transit_duration
    
    
    @property
    def b(self):
        """ Impact parameter b """
        if self._b is None:
            a = (self.semimajoraxis * u.AU).to(u.m)
            r_s = (self.star.radius * u.R_sun).to(u.m)
            i = self.inclination * u.degree
            self._b = (a / r_s) * np.cos(i)
            
        return self._b
    
    @property
    def impact_parameter(self):
        """ Thin wrapper for impact parameter """
        return self.b
    
    @property
    def transit_depth(self):
        """ Calculate the transit depth as a percentage """
        if self._transit_depth is None:
            r_s = (self.star.radius * u.R_sun).to(u.m)
            r_p = (self.radius * u.R_jup).to(u.m)
            self._transit_depth = (r_p / r_s)**2
            
        return self._transit_depth
    
    def in_transit(self, t1, with_times=False):
        """ Determine if the exoplanet was in transit at given time """
        if isinstance(t1, str):
            t1 = Time(t1)
        
        transittime = Time(star.planet.transittime, format='jd')
        period_delta = TimeDelta(star.planet.period, format='jd')
        
        num = int((t1 - transittime).sec // period_delta.sec)
        in_transit = False

        for n in range(num, num+2):
            midpoint = transittime + (n * period_delta)
            ingress = midpoint - (self.transit_duration * u.minute / 2)
            egress = midpoint + (self.transit_duration * u.minute / 2)

            in_transit = t1 >= ingress and t1 <= egress
            
            if in_transit:
                break
        
        if with_times:
            return (in_transit, Transit(ingress.isot, midpoint.isot, egress.isot))
        else:
            return in_transit
        
    def transits_in_range(self, t0, t1, num_of_transits=10):
        """ Determine if exoplanet transited in range of dates """
        if isinstance(t0, str):
            t0 = Time(t0)

        if isinstance(t1, str):
            t1 = Time(t1)

        transittime = Time(star.planet.transittime, format='jd')
        period_delta = TimeDelta(star.planet.period, format='jd')
        
        # Get nearest mid-transit point
        num = int((t0 - transittime).sec // period_delta.sec) - 1

        transits = list()
        
        while True:
            midpoint = transittime + (num * period_delta)
            ingress = midpoint - (self.transit_duration * u.minute / 2)
            egress = midpoint + (self.transit_duration * u.minute / 2)
            
            if ingress > t1:
                break
            
            transits.append(Transit(ingress.isot, midpoint.isot, egress.isot))
            
            num += 1


        return transits

### Start using the class

In [54]:
star = Star('TrES-3', file=oec_file)

In [57]:
star.planet.transittime

2455358.86636

In [58]:
star.planet.transit_duration

90.666291140986

In [59]:
print("Depth: {:.02%}".format(star.planet.transit_depth))

Depth: 2.49%


In [68]:
start_time = Time('2017-06-01')
end_time = Time('2017-06-30')

for t in star.planet.transits_in_range(start_time, end_time):
    print(t.ingress)

2017-05-29T18:50:53.319
2017-05-31T02:11:47.796
2017-06-01T09:32:42.274
2017-06-02T16:53:36.751
2017-06-04T00:14:31.228
2017-06-05T07:35:25.706
2017-06-06T14:56:20.183
2017-06-07T22:17:14.660
2017-06-09T05:38:09.138
2017-06-10T12:59:03.615
2017-06-11T20:19:58.092
2017-06-13T03:40:52.569
2017-06-14T11:01:47.047
2017-06-15T18:22:41.524
2017-06-17T01:43:36.001
2017-06-18T09:04:30.479
2017-06-19T16:25:24.956
2017-06-20T23:46:19.433
2017-06-22T07:07:13.911
2017-06-23T14:28:08.388
2017-06-24T21:49:02.865
2017-06-26T05:09:57.343
2017-06-27T12:30:51.820
2017-06-28T19:51:46.297


Search ours FTIS files to see if we captured a transit

In [61]:
fits_list = glob('/var/panoptes/images/fields/20170604T111430/*.fz', recursive=True)

In [62]:
fits_list.sort()
len(fits_list)

4

In [63]:
for f in fits_list:
    if f.endswith('.fz'):
        ext = 1
    else:
        ext = 0
        
    t1 = fits.getval(f, 'DATE-OBS', ext=ext)
    if star.planet.in_transit(t1):
        print(t1, f)